| Group             | 2025 Current $ | 2025 Current HC | 2025 Final $ | 2025 Final HC | 2026 Current $ | 2026 Current HC | 2026 Final $ | 2026 Final HC | 2027 Final $ | 2027 Final HC | 2028 Final $ | 2028 Final HC |
|------------------|----------------|------------------|--------------|----------------|----------------|------------------|--------------|----------------|--------------|----------------|--------------|----------------|
| **IM IT**        | $275,000,000   | 23,100           | $274,000,000 | 23,050         | $30,600,000    | 2,570            | $29,100,000  | 2,486          | $0           | 0              | $0           | 0              |
| Employees        | $149,000,000   | 11,700           | $148,000,000 | 11,740         | $16,600,000    | 1,310            | $15,100,000  | 1,226          | $0           | 0              | $0           | 0              |
| Contingent       | $126,000,000   | 11,400           | $126,000,000 | 11,340         | $14,000,000    | 1,260            | $14,000,000  | 1,260          | $0           | 0              | $0           | 0              |
| **Tech COO**     | $60,000,000    | 5,040            | $60,000,000  | 5,040          | $6,600,000     | 560              | $6,600,000   | 560            | $0           | 0              | $0           | 0              |
| Employees        | $54,000,000    | 4,230            | $54,000,000  | 4,230          | $6,000,000     | 470              | $6,000,000   | 470            | $0           | 0              | $0           | 0              |
| Contingent       | $6,000,000     | 810              | $6,000,000   | 810            | $630,000       | 90               | $630,000     | 90             | $0           | 0              | $0           | 0              |
| **Innovation**   | $148,000,000   | 13,590           | $148,000,000 | 13,590         | $16,500,000    | 1,510            | $16,500,000  | 1,510          | $0           | 0              | $0           | 0              |
| Employees        | $109,000,000   | 8,820            | $109,000,000 | 8,820          | $12,200,000    | 980              | $12,200,000  | 980            | $0           | 0              | $0           | 0              |
| Contingent       | $38,000,000    | 4,770            | $38,000,000  | 4,770          | $4,300,000     | 530              | $4,300,000   | 530            | $0           | 0              | $0           | 0              |


In [12]:
import json
from collections import defaultdict
from decimal import Decimal
from IPython.display import Markdown, display

# Load your JSON data
with open("input.json", "r") as f:
    data = json.load(f)

# Mapping IDs to labels
division_map = {item["i"]: item["l"] for item in data["drivers"]["DIVISION"]}
resource_class_map = {item["i"]: item["l"] for item in data["drivers"]["RESOURCE_CLASS"]}

# Store table data
table_data = defaultdict(lambda: defaultdict(lambda: {"$": 0.0, "HC": 0.0}))

# Populate data
for entry in data["data"]:
    drv = entry["drvs"]
    division_id = drv["DIVISION"]
    resource_class_id = drv["RESOURCE_CLASS"]
    year = entry["p"]

    group_label = division_map.get(division_id, f"Division {division_id}")
    class_label = resource_class_map.get(resource_class_id, f"Class {resource_class_id}")

    current_key = f"{year} Current"
    final_key = f"{year} Final"

    table_data[group_label][class_label][f"{current_key} $"] = Decimal(entry.get("d01", 0))
    table_data[group_label][class_label][f"{current_key} HC"] = Decimal(entry.get("f01", 0))
    table_data[group_label][class_label][f"{final_key} $"] = Decimal(entry.get("d12", 0))
    table_data[group_label][class_label][f"{final_key} HC"] = Decimal(entry.get("f12", 0))

# Define headers
headers = [
    "Group", "2025 Current $", "2025 Current HC", "2025 Final $", "2025 Final HC",
    "2026 Current $", "2026 Current HC", "2026 Final $", "2026 Final HC",
    "2027 Final $", "2027 Final HC", "2028 Final $", "2028 Final HC"
]

# Markdown table construction
rows = []
rows.append("| " + " | ".join(headers) + " |")
rows.append("|" + "|".join(["-" * len(h) for h in headers]) + "|")

def fmt_currency(val):
    return f"${val:.0f}" if val else "$0"

def fmt_int(val):
    return f"{int(val)}" if val else "0"

# Populate markdown rows
for division, resources in table_data.items():
    row = [f"**{division}**"]
    for year in ["2025", "2026"]:
        row.extend([
            fmt_currency(resources.get("Employees", {}).get(f"{year} Current $", 0)),
            fmt_int(resources.get("Employees", {}).get(f"{year} Current HC", 0)),
            fmt_currency(resources.get("Employees", {}).get(f"{year} Final $", 0)),
            fmt_int(resources.get("Employees", {}).get(f"{year} Final HC", 0))
        ])
    for year in ["2027", "2028"]:
        row.extend([
            fmt_currency(resources.get("Employees", {}).get(f"{year} Final $", 0)),
            fmt_int(resources.get("Employees", {}).get(f"{year} Final HC", 0))
        ])
    rows.append("| " + " | ".join(row) + " |")

    for resource in ["Employees", "Contingent"]:
        if resource in resources:
            row = [resource]
            for year in ["2025", "2026"]:
                row.extend([
                    fmt_currency(resources[resource].get(f"{year} Current $", 0)),
                    fmt_int(resources[resource].get(f"{year} Current HC", 0)),
                    fmt_currency(resources[resource].get(f"{year} Final $", 0)),
                    fmt_int(resources[resource].get(f"{year} Final HC", 0))
                ])
            for year in ["2027", "2028"]:
                row.extend([
                    fmt_currency(resources[resource].get(f"{year} Final $", 0)),
                    fmt_int(resources[resource].get(f"{year} Final HC", 0))
                ])
            rows.append("| " + " | ".join(row) + " |")

# Build markdown: THIS IS WHAT IS RETURNED TO THE AGENT BY THE TOOL
markdown = "\n".join(rows)

# Save to markdown file
with open("output_table.md", "w") as f:
    f.write(markdown)

print("✅ Markdown table saved to output_table.md")

✅ Markdown table saved to output_table.md
